# Plot paths samples by different path sampling mechanisms.

The goal of this notebook is to study the shaphes of the designs samples by the different strategies.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt


# Load Niklas data.
data_folder = "/home/cedric/PHD/Dev/VolcapySIAM/data/InversionDatas/stromboli_173018/"

data_coords = np.load(os.path.join(data_folder,"niklas_data_coords_corrected_final.npy"))
data_coords_inds_insurf = np.load(os.path.join(data_folder,"niklas_data_inds_insurf.npy"))[1:] # Remove base station.
surface_data_coords = np.load(os.path.join(data_folder,"surface_data_coords.npy"))

# Get the nodes coordinates. Remember the nodes store their corresponding data
# index in the 'ind' attribute.
xyz = surface_data_coords
mean = np.mean(xyz, axis=0)
std = np.std(xyz, axis=0)
xyz_norm = (xyz - mean) / std

# Scale the z coordinate.
xyz_norm[:, 2] = 0.5 * xyz_norm[:, 2]

from volcapy.graph.loading import load_Stromboli_graphs
graph_trails_insurf, graph_surface, accessibility_graph = load_Stromboli_graphs(data_folder)

In [ ]:
from volcapy.set_sampling import uniform_bernoulli_sample, uniform_size_uniform_sample
from volcapy.graph.cost_functions import symmetric_walking_cost_fn_edge

# Compute weights once and for all.
from volcapy.path_planning import weight_graph_with_cost
accessibility_graph = weight_graph_with_cost(accessibility_graph, symmetric_walking_cost_fn_edge)

base_node = list(graph_trails_insurf.nodes)[0]
n_starting_designs = 3

In [ ]:
%matplotlib inline
base_designs = uniform_size_uniform_sample(
    list(accessibility_graph.nodes), size=n_starting_designs,
    min_size=2, max_size=2, fixed_node=base_node)

# Get the shortest triangle connecting the points.
from volcapy.path_planning import solve_TSP
base_triangle_nodes = [solve_TSP(accessibility_graph, design_nodes, 'weight')[0] for design_nodes in base_designs]
base_triangle_paths = [
    [nx.shortest_path(accessibility_graph, base_triangle[i], base_triangle[i+1], 'weight') for i in range(len(base_triangle) - 1)]
    for base_triangle in base_triangle_nodes]

# Compute the costs.
base_triangle_costs = [
    (1 / 60**2) * np.sum([nx.shortest_path_length(accessibility_graph, base_triangle[i], base_triangle[i+1], 'weight') for i in range(len(base_triangle) - 1)])
    for base_triangle in base_triangle_nodes]

fig = plt.figure(figsize=(30, 10))

sizes = [300, 150, 70]
alphas = [1, 0.8, 0.5]
colors = ['blue', 'red', 'green']
plt.subplot(131)
# Plot the island.
plt.scatter(surface_data_coords[:, 0], surface_data_coords[:, 1], alpha=0.05)
for i, inds in enumerate(base_triangle_paths[0]):
    plt.scatter(surface_data_coords[inds, 0], surface_data_coords[inds, 1], s=sizes[i], alpha=alphas[1], color=colors[i])
plt.scatter(surface_data_coords[base_designs[0], 0], surface_data_coords[base_designs[0], 1], s=500, color='gold', marker='H')
plt.title("Cost: {:.1f} hrs".format(base_triangle_costs[0]))

plt.subplot(132)
# Plot the island.
plt.scatter(surface_data_coords[:, 0], surface_data_coords[:, 1], alpha=0.05)
for i, inds in enumerate(base_triangle_paths[1]):
    plt.scatter(surface_data_coords[inds, 0], surface_data_coords[inds, 1], s=sizes[i], alpha=alphas[1], color=colors[i])
plt.scatter(surface_data_coords[base_designs[1], 0], surface_data_coords[base_designs[1], 1], s=500, color='gold', marker='H')
plt.title("Cost: {:.1f} hrs".format(base_triangle_costs[1]))

plt.subplot(133)
# Plot the island.
plt.scatter(surface_data_coords[:, 0], surface_data_coords[:, 1], alpha=0.05)
for i, inds in enumerate(base_triangle_paths[2]):
    plt.scatter(surface_data_coords[inds, 0], surface_data_coords[inds, 1], s=sizes[i], alpha=alphas[1], color=colors[i])
plt.scatter(surface_data_coords[base_designs[2], 0], surface_data_coords[base_designs[2], 1], s=500, color='gold', marker='H')
plt.title("Cost: {:.1f} hrs".format(base_triangle_costs[2]))


plt.savefig("sampled_base_triangles.pdf")
plt.show()

In [ ]:
# Compute the costs.
base_triangle_costs = [
    (1 / 60**2) * np.sum([nx.shortest_path_length(accessibility_graph, base_triangle[i], base_triangle[i+1], 'weight') for i in range(len(base_triangle) - 1)])
    for base_triangle in base_triangle_nodes]

In [ ]:
base_triangle_costs